영문 원본 LLAVA - 허깅페이스에서 가져와서 pip 받을게 거의 없음

In [ ]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image

# 모델과 프로세서 로드
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")

# 이미지 불러오기
image = Image.open('/testimage/test.png')

# 이미지를 pixel_values로 변환 (전처리)
inputs = processor(image, return_tensors="pt")

# 캡션 생성
outputs = model.generate(**inputs)
caption = processor.decode(outputs[0], skip_special_tokens=True)

print("Generated caption:", caption)


OCR

In [ ]:
import easyocr

# EasyOCR 리더 초기화 (한글과 영어 지원)
reader = easyocr.Reader(['ko', 'en'])

# OCR 수행
result = reader.readtext('/testimage/test6.png')
text = '\n'.join([res[1] for res in result])

print("Generated OCR text:", text)


ocr, 캡션 합치기

In [ ]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import easyocr
import numpy as np

# BLIP 모델과 프로세서 로드
blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")

# EasyOCR 리더 초기화 (한글과 영어 지원)
ocr_reader = easyocr.Reader(['ko', 'en'])

# 이미지 불러오기
image = Image.open('/testimage/test.png').convert('RGB')

# 1. EasyOCR을 사용하여 이미지 내 텍스트 추출 (PIL 이미지를 numpy 배열로 변환)
ocr_result = ocr_reader.readtext(np.array(image))
ocr_text = ' '.join([res[1] for res in ocr_result])

# 2. BLIP를 사용하여 이미지 캡션 생성
inputs = blip_processor(image, return_tensors="pt")
outputs = blip_model.generate(**inputs)
caption = blip_processor.decode(outputs[0], skip_special_tokens=True)

# 3. OCR 결과와 BLIP 캡션 결합
combined_caption = f"OCR Text: {ocr_text}\nGenerated Caption: {caption}"

print(combined_caption)


한글 llava 시도

In [ ]:
# Load model directly
from transformers import AutoProcessor, AutoModelForCausalLM

processor = AutoProcessor.from_pretrained("KBNIT/KoLLaVA-1.5v-kolon-v1.6")
model = AutoModelForCausalLM.from_pretrained("KBNIT/KoLLaVA-1.5v-kolon-v1.6")

# 이미지 불러오기
image = Image.open('test4.png')

# 이미지를 pixel_values로 변환 (전처리)
inputs = processor(image, return_tensors="pt")

# 캡션 생성
outputs = model.generate(**inputs)
caption = processor.decode(outputs[0], skip_special_tokens=True)

print("Generated caption:", caption)

허깅페이스 api 호출

In [4]:
from dotenv import load_dotenv
import os

# .env 파일 로드
load_dotenv()

# 환경 변수에서 Hugging Face API 토큰 가져오기
api_token = os.getenv('HUGGING_FACE_API')

In [ ]:
import requests
from PIL import Image
import base64

# 이미지 파일을 읽어서 base64로 변환
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

# Hugging Face API 호출 함수
def query(image_path):
    API_URL = "https://api-inference.huggingface.co/models/Salesforce/blip-image-captioning-large"
    headers = {"Authorization": f"Bearer {api_token}"}  # Hugging Face API 토큰
    image_data = encode_image(image_path)

    # API 요청 데이터 (이미지를 base64로 변환 후 전송)
    payload = {
        "inputs": {
            "image": image_data
        }
    }

    # API 호출
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

# 이미지 파일 경로
image_path = 'test4.png'

# 결과 출력
result = query(image_path)
print("Generated caption:", result)
